<a href="https://colab.research.google.com/github/renanas/MachineLearning_Study/blob/main/Classificacao_Churn_Utilizando_algumas_tecnicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificao de Churn
Neste estudo vamos ver algumas tecnicas de prever o churn.
Pegamos da seguinte base:

E vamos estudar KNeighborsClassifier, BernoulliNB e DecisionTreeClassifier.
No final vamos comparar qual tecnica se sobre sai e qual iremos utilizar para o final. Também utilizaremos algumas metricas para analisar qual se sai melhor, utilizando as tecnicas de acuracia, precisão e recall.


In [1]:
import pandas as pd

In [2]:
uri = "/content/Customer-Churn.csv"
dados = pd.read_csv(uri)
dados

,Maior65Anos,Conjuge,Dependentes,MesesDeContrato,TelefoneFixo,VariasLinhasTelefonicas,ServicoDeInternet,SegurancaOnline,BackupOnline,SeguroNoDispositivo,SuporteTecnico,TVaCabo,StreamingDeFilmes,TipoDeContrato,PagamentoOnline,FormaDePagamento,ContaMensal,Churn
0,0,Sim,Nao,1,Nao,SemServicoTelefonico,DSL,Nao,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequeDigital,29.85,Nao
1,0,Nao,Nao,34,Sim,Nao,DSL,Sim,Nao,Sim,Nao,Nao,Nao,UmAno,Nao,ChequePapel,56.95,Nao
2,0,Nao,Nao,2,Sim,Nao,DSL,Sim,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequePapel,53.85,Sim
3,0,Nao,Nao,45,Nao,SemServicoTelefonico,DSL,Sim,Nao,Sim,Sim,Nao,Nao,UmAno,Nao,DebitoEmConta,42.30,Nao
4,0,Nao,Nao,2,Sim,Nao,FibraOptica,Nao,Nao,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequeDigital,70.70,Sim
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,Sim,Sim,24,Sim,Sim,DSL,Sim,Nao,Sim,Sim,Sim,Sim,UmAno,Sim,ChequePapel,84.80,Nao
7039,0,Sim,Sim,72,Sim,Sim,FibraOptica,Nao,Sim,Sim,Nao,Sim,Sim,UmAno,Sim,CartaoDeCredito,103.20,Nao
7040,0,Sim,Sim,11,Nao,SemServicoTelefonico,DSL,Sim,Nao,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequeDigital,29.60,Nao
7041,1,Sim,Nao,4,Sim,Sim,FibraOptica,Nao,Nao,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequePapel,74.40,Sim


In [3]:
dados.shape

(7043, 18)

In [4]:
traducao_dicionario = {
    'Nao': 0,
    'Sim': 1
}
dadosmodificados = dados[['Conjuge', 'Dependentes', 'TelefoneFixo', 'PagamentoOnline', 'Churn']].replace(traducao_dicionario)
dadosmodificados.head()

,Conjuge,Dependentes,TelefoneFixo,PagamentoOnline,Churn
0,1,0,0,1,0
1,0,0,1,0,0
2,0,0,1,1,1
3,0,0,0,0,0
4,0,0,1,1,1


In [5]:
dummie_dados = pd.get_dummies(dados.drop(['Conjuge', 'Dependentes', 'TelefoneFixo', 'PagamentoOnline', 'Churn'],
                                         axis=1))
dados_final = pd.concat([dadosmodificados, dummie_dados], axis=1)

In [6]:
dados_final.head()

,Conjuge,Dependentes,TelefoneFixo,PagamentoOnline,Churn,Maior65Anos,MesesDeContrato,ContaMensal,VariasLinhasTelefonicas_Nao,VariasLinhasTelefonicas_SemServicoTelefonico,...,StreamingDeFilmes_Nao,StreamingDeFilmes_SemServicoDeInternet,StreamingDeFilmes_Sim,TipoDeContrato_DoisAnos,TipoDeContrato_Mensalmente,TipoDeContrato_UmAno,FormaDePagamento_CartaoDeCredito,FormaDePagamento_ChequeDigital,FormaDePagamento_ChequePapel,FormaDePagamento_DebitoEmConta
0,1,0,0,1,0,0,1,29.85,0,1,...,1,0,0,0,1,0,0,1,0,0
1,0,0,1,0,0,0,34,56.95,1,0,...,1,0,0,0,0,1,0,0,1,0
2,0,0,1,1,1,0,2,53.85,1,0,...,1,0,0,0,1,0,0,0,1,0
3,0,0,0,0,0,0,45,42.30,0,1,...,1,0,0,0,0,1,0,0,0,1
4,0,0,1,1,1,0,2,70.70,1,0,...,1,0,0,0,1,0,0,1,0,0


In [7]:
dados_final.shape

(7043, 39)

In [8]:
pd.set_option('display.max_columns', 39)
dados_final.head()

,Conjuge,Dependentes,TelefoneFixo,PagamentoOnline,Churn,Maior65Anos,MesesDeContrato,ContaMensal,VariasLinhasTelefonicas_Nao,VariasLinhasTelefonicas_SemServicoTelefonico,VariasLinhasTelefonicas_Sim,ServicoDeInternet_DSL,ServicoDeInternet_FibraOptica,ServicoDeInternet_Nao,SegurancaOnline_Nao,SegurancaOnline_SemServicoDeInternet,SegurancaOnline_Sim,BackupOnline_Nao,BackupOnline_SemServicoDeInternet,BackupOnline_Sim,SeguroNoDispositivo_Nao,SeguroNoDispositivo_SemServicoDeInternet,SeguroNoDispositivo_Sim,SuporteTecnico_Nao,SuporteTecnico_SemServicoDeInternet,SuporteTecnico_Sim,TVaCabo_Nao,TVaCabo_SemServicoDeInternet,TVaCabo_Sim,StreamingDeFilmes_Nao,StreamingDeFilmes_SemServicoDeInternet,StreamingDeFilmes_Sim,TipoDeContrato_DoisAnos,TipoDeContrato_Mensalmente,TipoDeContrato_UmAno,FormaDePagamento_CartaoDeCredito,FormaDePagamento_ChequeDigital,FormaDePagamento_ChequePapel,FormaDePagamento_DebitoEmConta
0,1,0,0,1,0,0,1,29.85,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0
1,0,0,1,0,0,0,34,56.95,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0
2,0,0,1,1,1,0,2,53.85,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0
3,0,0,0,0,0,0,45,42.30,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,1
4,0,0,1,1,1,0,2,70.70,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0


Utilizando o KNN para classificação.

In [9]:
X_new_amostra = [[
 0, 0, 1, 1, 0, 0, 39.9, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1
]]

In [10]:
X = dados_final.drop('Churn', axis = 1)
y = dados_final['Churn']

In [11]:
from sklearn.preprocessing import StandardScaler

norm = StandardScaler()

X_normalizado = norm.fit_transform(X)
X_normalizado

array([[ 1.03453023, -0.65401193, -3.05401039, ...,  1.40641839,
        -0.54480692, -0.5298852 ],
       [-0.96662231, -0.65401193,  0.32743831, ..., -0.71102597,
         1.83551265, -0.5298852 ],
       [-0.96662231, -0.65401193,  0.32743831, ..., -0.71102597,
         1.83551265, -0.5298852 ],
       ...,
       [ 1.03453023,  1.5290241 , -3.05401039, ...,  1.40641839,
        -0.54480692, -0.5298852 ],
       [ 1.03453023, -0.65401193,  0.32743831, ..., -0.71102597,
         1.83551265, -0.5298852 ],
       [-0.96662231, -0.65401193,  0.32743831, ..., -0.71102597,
        -0.54480692,  1.88720123]])

In [12]:
X_new_amostra_normalizado = norm.transform(pd.DataFrame(X_new_amostra, columns= X.columns))
X_new_amostra_normalizado

array([[-0.96662231, -0.65401193,  0.32743831,  0.8297975 , -0.43991649,
        -1.31816495, -0.82630172,  1.03806605, -0.32743831, -0.85417615,
        -0.72373975,  1.12910177, -0.52592737,  1.00669571, -0.52592737,
        -0.63393298, -0.88361972, -0.52592737,  1.3782406 ,  1.12942721,
         1.90140322, -0.72396752, -0.98632101,  1.90140322, -0.63943864,
         1.2273572 , -0.52592737, -0.79013171, -0.80874158, -0.52592737,
         1.25617071, -0.56297505, -1.1059698 ,  1.94458183, -0.52504733,
        -0.71102597, -0.54480692,  1.88720123]])

Utilizando a distancia Euclidiana

In [13]:
import numpy as np

a = X_new_amostra_normalizado
b = X_normalizado[0]

np.sqrt(np.sum(np.square(a - b)) )

9.169282435541762

Essa é a distancia do novo registro com o cliente 0. Feito na mão, agora vamos implementar para calcular a distancia para todos os clientes

In [14]:
from sklearn.model_selection import train_test_split

X_treino, X_teste, y_treino, y_teste = train_test_split(X_normalizado, y, test_size=0.3, random_state=123)

In [15]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(metric = 'euclidean')
knn.fit(X_treino, y_treino)

KNeighborsClassifier(metric='euclidean')

In [16]:
predito_knn = knn.predict(X_teste)
predito_knn

array([1, 0, 0, ..., 0, 0, 0])

In [17]:
from sklearn.naive_bayes import BernoulliNB

In [22]:
bnb = BernoulliNB(binarize=0.44)

In [23]:
bnb.fit(X_treino, y_treino)

BernoulliNB(binarize=0.44)

In [24]:
predito_bnb = bnb.predict(X_teste)

In [25]:
predito_bnb

array([1, 0, 0, ..., 0, 1, 0])

In [26]:
from sklearn.tree import DecisionTreeClassifier

In [27]:
dtc = DecisionTreeClassifier(criterion='entropy', random_state=42)

In [28]:
dtc.fit(X_treino, y_treino)

DecisionTreeClassifier(criterion='entropy', random_state=42)

In [29]:
dtc.feature_importances_

array([0.03168742, 0.01862901, 0.00064085, 0.02675693, 0.0208788 ,
       0.2056003 , 0.29546501, 0.0061786 , 0.00273035, 0.00747536,
       0.        , 0.04908356, 0.        , 0.01609395, 0.        ,
       0.00722833, 0.01220706, 0.        , 0.01248286, 0.01138295,
       0.0006673 , 0.0095669 , 0.01841631, 0.        , 0.00650511,
       0.00385849, 0.        , 0.00666929, 0.00517146, 0.        ,
       0.00665371, 0.00473251, 0.15803541, 0.00348686, 0.01414378,
       0.01601964, 0.01064163, 0.01091026])

In [30]:
predito_ArvoreDecisao = dtc.predict(X_teste)

In [31]:
predito_ArvoreDecisao

array([1, 0, 0, ..., 1, 0, 0])

In [32]:
from sklearn.metrics import confusion_matrix

In [33]:
print(confusion_matrix(y_teste, predito_knn))

[[1305  218]
 [ 317  273]]


In [35]:
print(confusion_matrix(y_teste, predito_bnb))

[[1059  464]
 [ 120  470]]


In [36]:
print(confusion_matrix(y_teste, predito_ArvoreDecisao))

[[1249  274]
 [ 290  300]]


A primeria metrica para vermos como os nossos modelos estão bons é a acuracia. Ela vai medir o quanto cada modelo acerto no total.

In [37]:
from sklearn.metrics import accuracy_score

In [38]:
print(accuracy_score(y_teste, predito_knn))

0.7468054898248935


In [40]:
print(accuracy_score(y_teste, predito_bnb))

0.7236157122574538


In [41]:
print(accuracy_score(y_teste, predito_ArvoreDecisao))

0.7330809275911027


A proxima é a precisão, ela calcula quantos Verdadeiros positivos foram calculados de forma correta. Quando é para ser Verdadeiro e ela previu que deu Verdadeiro.

In [42]:
from sklearn.metrics import precision_score

In [45]:
print(precision_score(y_teste, predito_knn))

0.5560081466395111


In [48]:
print(precision_score(y_teste, predito_bnb))

0.5032119914346895


In [49]:
print(precision_score(y_teste, predito_ArvoreDecisao))

0.5226480836236934


E a ultima metrica é o recall. O recall mede os que realmente deram positivos, somando os Verdeiro Positivo, aquele que era verdadeiro e previu verdadeiro, com o falso negativo, aquele que era para ser falso e previu como falso.

In [50]:
from sklearn.metrics import recall_score

In [52]:
print(recall_score(y_teste, predito_knn))

0.46271186440677964


In [53]:
print(recall_score(y_teste, predito_bnb))

0.7966101694915254


In [54]:
print(recall_score(y_teste, predito_ArvoreDecisao))

0.5084745762711864


Cada modelo pode ser melhor do que o outro dependendo da metrica e qual é o melhor?

Vamos considerar que o melhor é a Recall. Pq precisamos ser mais acertivos em prever os que estão realmente saindo com os que não estão saindo, assim podemos agir/atuar com ações apenas naqueles que estão saindo, confirmando o churn.

In [56]:
print('Modelo Knn: ', recall_score(y_teste, predito_knn))
print('Modelo Bernoulli de Naive Bayes: ', recall_score(y_teste, predito_bnb))
print('Modelo Arvore de Decisao: ',recall_score(y_teste, predito_ArvoreDecisao))

Modelo Knn:  0.46271186440677964
Modelo Bernoulli de Naive Bayes:  0.7966101694915254
Modelo Arvore de Decisao:  0.5084745762711864


O melhor modelo é o Bernoulli de Naive Bayes e utilizaremos ele para a nossa aplicação.